In [1]:
# By no means a clean and organized parser.

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
uri = "https://forums.flightsimulator.com/t/how-to-graphics-settings-and-performance-guide-3-16-2021/132407"

data = requests.get(uri)

In [3]:
soup = BeautifulSoup(data.content, 'html.parser')

In [9]:
parsed = []
started = False
for detail in soup.find_all('details'):
    if 'anti alias' in detail.find_next().text.lower():
        started = True

    if started:
        setting_name = detail.find_next().text.lower().strip()
        text = detail.text
        tokens = ['recommended setting', 'performance impact', 'visual impact']
        tokens_in_text = [s for s in tokens if s in text.lower()]
        # print('tokens_in_text', tokens_in_text)
        current_line = 0
        index_dict = dict(zip(tokens, [-1] * len(tokens)))
        lines = text.split('\n')

        for i, line in enumerate(lines):
            for t in tokens_in_text:
                if t + ':' in line.lower():
                    index_dict[t] = i
        
        names = sorted(index_dict, key=lambda x: index_dict[x])
        values = [index_dict[k] for k in names]
        setting_data = {'setting': setting_name}

        

        for i, key in enumerate(names):
            if key in tokens_in_text:
                start = index_dict[key]
                if i < len(names) - 1:
                    end = index_dict[names[i+1]]
                else:
                    end = -1
                
                setting_data[key] = '\n'.join(lines[start:end])
            else:
                setting_data[key] = ''

        parsed.append(setting_data)
        
df = pd.DataFrame(parsed)
df['recommended setting'] = df['recommended setting'].map(lambda x: x.replace('Recommended Setting: ', '').strip().replace('\n', ' '))
df['performance impact'] = df['performance impact'].map(lambda x: x.replace('Performance Impact:\n', '').strip().replace('\n', ' '))
df['visual impact'] = df['visual impact'].map(lambda x: x.replace('Visual Impact:\n', '').strip().replace('\n', ' '))

with open('MSFS2020_settings_guide.html', 'w') as out:
    out.write(df.to_html())

with open('MSFS2020_settings_guide.md', 'w') as out:
    out.write(df.to_markdown())